In [ ]:
!pip install transformers

# Import MLM model and tokenizer

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM

In [ ]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Test

In [ ]:
model.eval() # model to eval mode
text = 'I want to [MASK] the car because it is cheap . Malaria is a [MASK] disease.'
ids =  tokenizer.encode(text, return_tensors='pt')

In [ ]:
ids

tensor([[  101,  1045,  2215,  2000,   103,  1996,  2482,  2138,  2009,  2003,
         10036,  1012, 19132,  2003,  1037,   103,  4295,  1012,   102]])

In [ ]:
import torch
mask_token_index = torch.where(ids == tokenizer.mask_token_id)[1]
mask_token_index

tensor([ 4, 15])

In [ ]:
with torch.no_grad():
    output = model(ids)

In [ ]:
mask_token_logits = output.logits[0, mask_token_index, :]

In [ ]:
torch.topk(mask_token_logits, 5, dim=1).indices.tolist()

[[5271, 2202, 4965, 3298, 9278], [3809, 2691, 9252, 5133, 5536]]

In [ ]:
top5s = torch.topk(mask_token_logits, 5, dim=1).indices.tolist()
for top5 in top5s:
  predict_words = [tokenizer.decode([id]) for id in top5]
  print(predict_words)


['sell', 'take', 'buy', 'drive', 'rent']
['serious', 'common', 'deadly', 'tropical', 'terminal']


# Test on vnmese

In [ ]:
!pip3 install py_vncorenlp

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.9 MB/s eta 0:00:00
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4305 sha256=1b3a47ed871cd7cbf4c87332d0399287e5e5bac8d6ca7f8355c42ae62bf7fa19
  Stored in directory: /root/.cache/pip/wheels/d5/d9/bf/62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827
Successfully built py_vncorenlp


In [ ]:
!mkdir vncorenlp

In [ ]:
import py_vncorenlp
py_vncorenlp.download_model(save_dir='/content/vncorenlp')
model = py_vncorenlp.VnCoreNLP(save_dir='/content/vncorenlp')


In [ ]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-multilingual-cased')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
sentence = "Ngày hôm nay, có một cô gái [MASK] và làm toán với tôi. Tôi rất [MASK]"
def masked_word_seg(sentence):
  texts = model.word_segment(sentence)
  texts = [text.replace('[ MASK ]', '[MASK]') for text in texts]
  return " ".join(texts)

seg_sentence = masked_word_seg(sentence)

In [ ]:
unmasker(seg_sentence)

[[{'score': 0.19624343514442444,
   'token': 12002,
   'token_str': 'đến',
   'sequence': '[CLS] Ngày hôm _ nay, có một cô gái đến và làm toán với tôi. Tôi rất [MASK] [SEP]'},
  {'score': 0.06345115602016449,
   'token': 117,
   'token_str': ',',
   'sequence': '[CLS] Ngày hôm _ nay, có một cô gái, và làm toán với tôi. Tôi rất [MASK] [SEP]'},
  {'score': 0.04962293431162834,
   'token': 16895,
   'token_str': 'đi',
   'sequence': '[CLS] Ngày hôm _ nay, có một cô gái đi và làm toán với tôi. Tôi rất [MASK] [SEP]'},
  {'score': 0.03792138397693634,
   'token': 22072,
   'token_str': 'nói',
   'sequence': '[CLS] Ngày hôm _ nay, có một cô gái nói và làm toán với tôi. Tôi rất [MASK] [SEP]'},
  {'score': 0.036164410412311554,
   'token': 43094,
   'token_str': 'bạn',
   'sequence': '[CLS] Ngày hôm _ nay, có một cô gái bạn và làm toán với tôi. Tôi rất [MASK] [SEP]'}],
 [{'score': 0.41027048230171204,
   'token': 119,
   'token_str': '.',
   'sequence': '[CLS] Ngày hôm _ nay, có một cô gái [MAS

# Training